<a href="https://colab.research.google.com/github/alessandrocma/ProjetoImersaoAluraIAGoogle/blob/main/Aula5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q -U google-generativeai

In [57]:
# Import the Python SDK
import google.generativeai as genai
import numpy as np
import pandas as pd
# Used to securely store your API key
from google.colab import userdata

#Definindo a Google API Key
GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

#Selecionando o modelo do Gemini
#model_gemini = genai.GenerativeModel('gemini-pro')
model_gemini = genai.GenerativeModel('models/gemini-1.5-pro-latest')


In [58]:
#Listando os embeddings disponíveis
for i in genai.list_models():
  if 'embedContent' in i.supported_generation_methods:
    print(i.name)

#Submetendo um embedding
title = "Titulo"
text = "Hello World"
model_embedding="models/embedding-001"
result = genai.embed_content(model=model_embedding, content=text, title=title, task_type="RETRIEVAL_DOCUMENT")
print(result["embedding"])

models/embedding-001
models/text-embedding-004
[0.06799325, -0.02992038, -0.037183788, -0.00097259856, 0.054602183, 0.03546737, 0.039974865, -0.0025020407, -0.014234407, 0.040933635, 0.008256478, 0.024493827, -0.035463113, -0.013894533, 0.0015149742, 0.026969431, 0.0142610455, -0.008454018, -0.016449692, 0.0052818167, 0.012070838, 0.0018761471, -0.03912061, -0.041887194, 0.03404923, 0.0077958927, 0.010473964, -0.045959007, -0.015656486, 0.028983088, -0.01676534, -0.01461522, -0.03622627, 0.025461135, 0.027102923, -0.057657927, -0.007638016, 0.020478189, 0.00084335887, -0.013833034, 0.0021403446, -0.07100391, -0.06017263, -0.0028103627, 0.03948946, -0.013817518, -0.03194691, 0.01772756, 0.008971009, -0.072825156, 0.04015163, 0.001035276, 0.091220856, -0.028226677, 0.0134277325, -0.03344625, 0.0572884, -0.0053406707, -0.062753566, 0.010112223, -0.0065343515, 0.008640357, 0.036954843, 0.0261679, -0.021791259, -0.050069664, -0.066978656, 0.02558158, 0.021928951, -0.005030221, -0.0065710363

In [59]:
#Definindo documentos
DOCUMENT1 = {
    "Titulo":"Golden",
    "Conteudo":"Goldens são muito espertos",
}

DOCUMENT2 = {
    "Titulo":"Terapia",
    "Conteudo":"Terapias integrativas ajudam muito",
}

documents = [DOCUMENT1, DOCUMENT2]

#Criando um Data Frame
df = pd.DataFrame(documents)
df.columns=["Titulo","Conteudo"]
df


,Titulo,Conteudo
0,Golden,Goldens são muito espertos
1,Terapia,Terapias integrativas ajudam muito


In [60]:
#Retornando os embeddings de um documento
def embed_fn(title, text,model):
  return genai.embed_content(model=model, content=text, title=title, task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [61]:
#Adicionando uma coluna no Data Frame
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"],model_embedding),axis=1)
df

,Titulo,Conteudo,Embeddings
0,Golden,Goldens são muito espertos,"[0.015008322, -0.034960166, -0.049137816, 0.00..."
1,Terapia,Terapias integrativas ajudam muito,"[0.050589584, -0.025871804, -0.044818606, 0.01..."


In [62]:
def gerar_e_buscar_consulta(consulta, base,model):
  embedding_da_consulta = genai.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]),embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [67]:
consulta = "Qual e a melhor terapia para mim?"
#consulta = "Qual é o cachorro mais esperto?"
resultado = gerar_e_buscar_consulta(consulta=consulta, base=df, model=model_embedding)
print(resultado)

Terapias integrativas ajudam muito


In [71]:
prompt = f"Reescreva esse texto de uma forma mais amavel adicionando informações com 100% de certeza de estarem corretas: {resultado}"
response = model_gemini.generate_content(prompt)
print(response.text)

As terapias integrativas podem ser um complemento valioso ao tratamento tradicional, oferecendo diversas maneiras de cuidar do bem-estar físico, emocional e mental. 

**É importante ressaltar que as terapias integrativas não substituem o tratamento médico convencional.**  

No entanto,  **comprovadamente**,  podem auxiliar no manejo de sintomas, redução do estresse e ansiedade, melhora do sono, alívio de dores e promoção do bem-estar geral. 

**Alguns exemplos de terapias integrativas com benefícios comprovados por estudos científicos são:**

* **Yoga:**  Aumenta a flexibilidade, força e equilíbrio, além de reduzir o estresse e a ansiedade.
* **Meditação:**  Promove relaxamento profundo, reduz a pressão arterial e melhora o foco e a concentração.
* **Acupuntura:**  Alivia dores crônicas, náuseas e outros sintomas, através da aplicação de agulhas em pontos específicos do corpo.
* **Massagem Terapêutica:**  Reduz a tensão muscular, melhora a circulação sanguínea e promove relaxamento.

A